In [1]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# LOCATION = "../pressure_injury_data" 
LOCATION = "../new_pressure_injury_data" 


admission_df = pd.read_csv(os.path.join(LOCATION, "ADMISSION.csv"))
disch_df = pd.read_csv(os.path.join(LOCATION, "DISCH_DX.csv"))

### Add missing POA data from 2011-2015

In [4]:
missing_poa_df = pd.read_csv(os.path.join(LOCATION, "2011_2015_pressure_sores_poa_for_delivery.csv"))
missing_poa_df = missing_poa_df.head(missing_poa_df.POA.value_counts()[1])

In [5]:
def add_missing_poa(x):
    indices = admission_df.query(f"DE_ID_PAT_ID == '{x.DE_ID_PAT_ID}' & ENCOUNTER_COUNTER == {x.ENCOUNTER_COUNTER}").index
    assert len(indices) == 1
    admission_df.at[indices[0],"POA"] = 1
    
missing_poa_df.apply(lambda x: add_missing_poa(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1042    None
1043    None
1044    None
1045    None
1046    None
Length: 1047, dtype: object

In [6]:
def check_L89(row):
    for val in row:
        if isinstance(val, str) and "L89" in val:
            return 1
    return 0

In [7]:
encounters_grouped = admission_df.groupby(['DE_ID_PAT_ID', 'ENCOUNTER_COUNTER'])[["WOUND_CARE_NURSE_CONSULT", "WOUND_CARE_TEAM_CONSULT", "POA"]].max()

#### Selection criteria:
- Wound care team and wound care nurse consult
- No POA
- Yes L89

Yields only 1 patients consult AND wound care nurse consult
- Only one patient

In [27]:
def positive_to_json(boolean_array, path):
    assert isinstance(boolean_array, pd.Series)
    pos_locs = boolean_array
    pos_locs = pos_locs[pos_locs].index.values
    
    if False or input(f"Save to '{path}'? \n(Y/[N])") == "Y":
        with open(path, 'w') as f:
            json.dump(pos_locs.tolist(), f)
        print("Saved.")
    else:
        print("Not saved.")

    
    
    return path

path = os.path.join(LOCATION, "new_indices_of_positive.json")
positive_to_json((encounters_grouped.POA == 0) & (encounters_grouped.L89_ON_DISCHARGE == 1), path)

Save to '../new_pressure_injury_data/new_indices_of_positive.json'? 
(Y/[N]) n


Not saved.


'../new_pressure_injury_data/new_indices_of_positive.json'